In [1]:
# import necessary modules
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# training data
train = pd.read_csv("C:/Users/manas/train_final.csv")
print('training dataset shape:', train.shape)
#print(app_train.columns)
test = pd.read_csv("C:/Users/manas/test_final.csv")
print('test dataset shape:', test.shape)

training dataset shape: (307511, 1422)
test dataset shape: (48744, 1421)


## Model Prediction

In [5]:
## substitute missing value with imputer then scaling with MinMaxScaler
from sklearn.preprocessing import MinMaxScaler, Imputer
train_labels = train['TARGET']
train = train.drop(columns=['TARGET'])
features = list(train.columns)
test = test.copy()
imputer = Imputer(strategy='median')
scaler = MinMaxScaler(feature_range = (0, 1))
imputer.fit(train)
train = imputer.transform(train)
test = imputer.transform(test)
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)
print(train.shape, test.shape)

(307511, 1421) (48744, 1421)


In [13]:
train = pd.DataFrame(train, columns = features)
test = pd.DataFrame(test, columns = features)
train.head()

,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,...,EXT_SOURCE_2^3,EXT_SOURCE_2^2 EXT_SOURCE_3,EXT_SOURCE_2^2 DAYS_BIRTH,EXT_SOURCE_2 EXT_SOURCE_3^2,EXT_SOURCE_2 EXT_SOURCE_3 DAYS_BIRTH,EXT_SOURCE_2 DAYS_BIRTH^2,EXT_SOURCE_3^3,EXT_SOURCE_3^2 DAYS_BIRTH,EXT_SOURCE_3 DAYS_BIRTH^2,DAYS_BIRTH^3
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.626563,0.541836,0.974037,0.541837,0.538356,0.575199,0.583191,0.992715,0.551953,0.972706
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.763635,0.328873,0.742370,0.874526,0.778853,0.707433,0.000000,0.335487,0.302298,0.725543
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.721095,0.657662,0.766394,0.698595,0.373740,0.730817,0.807472,0.598178,0.766427,0.585062
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.784717,0.309615,0.680881,0.889726,0.821909,0.759889,0.000000,0.246700,0.234152,0.587909
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.636062,0.480765,0.917601,0.713067,0.689623,0.666656,0.000000,0.209957,0.203454,0.520494


In [11]:
app_train = pd.read_csv('C:/Users/manas/Desktop/Springboard/Capstone ideas/Home credit default risk/Data/application_train.csv')
app_test = pd.read_csv('C:/Users/manas/Desktop/Springboard/Capstone ideas/Home credit default risk/Data/application_test.csv')

In [14]:
train['SK_ID_CURR'] = app_train['SK_ID_CURR']
test['SK_ID_CURR'] = app_test['SK_ID_CURR']
print('training dataset shape:', train.shape)
print('test dataset shape:', test.shape)

training dataset shape: (307511, 1421)
test dataset shape: (48744, 1421)


Now the data is ready to be used for model prediction

In [16]:
# ## LightGBM
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

def model(features, train_labels, test_features, n_folds = 5):
    
       
    # Extract the ids
    train_id = features['SK_ID_CURR']
    test_id = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = train_labels
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'],
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_id, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [17]:
submission, fi, metrics = model(train, train_labels, test)
print(metrics)

Training Data Shape:  (307511, 1420)
Testing Data Shape:  (48744, 1420)
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.783442	valid's binary_logloss: 0.531643	train's auc: 0.83227	train's binary_logloss: 0.518788
[400]	valid's auc: 0.78479	valid's binary_logloss: 0.504649	train's auc: 0.86956	train's binary_logloss: 0.480325
Early stopping, best iteration is:
[411]	valid's auc: 0.784974	valid's binary_logloss: 0.503182	train's auc: 0.871454	train's binary_logloss: 0.478251
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.780963	valid's binary_logloss: 0.533257	train's auc: 0.832822	train's binary_logloss: 0.518454
[400]	valid's auc: 0.7825	valid's binary_logloss: 0.50689	train's auc: 0.869646	train's binary_logloss: 0.480099
Early stopping, best iteration is:
[325]	valid's auc: 0.782789	valid's binary_logloss: 0.515675	train's auc: 0.857538	train's binary_logloss: 0.493221
Training until validation scores don't imp

A thorough cleaning and inclusion of other tables has improved the score from 0.7657 to 0.7853.

In [18]:
fi_sorted = fi.sort_values('importance', ascending=False).reset_index()
fi_sorted.head() 

,index,feature,importance
0,1383,CREDIT_TERM,334.4
1,1307,AMT_ANNUITY,164.8
2,1294,client_installments_AMT_PAYMENT_min_sum,142.0
3,1323,EXT_SOURCE_1_x,114.0
4,1306,AMT_CREDIT,99.0
